<img src="https://github.com/nicholasmetherall/digital-earth-pacific-macblue-activities/blob/main/attachments/images/DE_Pacific_banner.JPG?raw=true" width="900"/>

Figure 1.1.a. Jupyter environment + Python notebooks

# Digital Earth Pacific Notebook 1 prepare postcard and load data to csv

The objective of this notebook is to prepare a geomad postcard for your AOI (masking, scaling and loading additional band ratios and spectral indices) and sampling all the datasets into a csv based on your training data geodataframe.

## Step 1.1: Configure the environment

In [1]:
import os
from datetime import datetime
from shapely.geometry import Polygon
from shapely import box
from pyproj import CRS 
import folium
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio as rio
import xarray as xr
import rioxarray
from ipyleaflet import basemaps
from numpy.lib.stride_tricks import sliding_window_view
import pystac_client
from dask.distributed import Client as DaskClient
from odc.stac import load, configure_s3_access
import planetary_computer
from odc.stac import load
from pystac.client import Client
from skimage.feature import graycomatrix, graycoprops
from utils import load_data, scale, calculate_band_indices, apply_mask, mask_land, mask_deeps, mask_elevation, all_masks, glcm_features, do_prediction

In [2]:
# Reload scripts and imports
%load_ext autoreload
%autoreload 2

In [3]:
# Predefined variable for title and version

# Enter your initials
initials = "nm"

# Enter your site name
site = "serua_2"

# Date
date = datetime.now()

# Make a clean version string
version = f"{initials}-{site}-{date.strftime('%d%m%Y')}"
print(version)

nm-serua_2-26072025


## Step 1.2: Configure STAC access and search parameters

In [4]:
catalog = "https://stac.digitalearthpacific.org"
client = Client.open(catalog)

In [ ]:
postcards = postcards.drop(columns=['coastal_class', 'fid'])

In [ ]:
postcards

In [6]:
# training = gpd.read_file(f"training-data/{version}_postcard.geojson")
# training = training.to_crs("EPSG:4326")
# min_lon, min_lat, max_lon, max_lat = training.total_bounds

# bbox = [min_lon, min_lat, max_lon, max_lat]

In [7]:
datetime = "2024"

items = client.search(
    collections=["dep_s2_geomad"],
    datetime=datetime,
    bbox=bbox
).item_collection()

print(f"Found {len(items)} items in for {datetime}")

Found 1 items in for 2024


In [8]:
measurements = ["nir", "red", "blue", "green", "emad", "smad", "bcmad", "green", "nir08", "nir09", "swir16", "swir22", "coastal", "rededge1", "rededge2", "rededge3"]
data = load_data(
    items,
    measurements,
    bbox,
)
    
# Now you can use the 'data' variable
print(data)

<xarray.Dataset> Size: 1MB
Dimensions:      (y: 133, x: 253, time: 1)
Coordinates:
  * y            (y) float64 1kB -2.052e+06 -2.052e+06 ... -2.054e+06 -2.054e+06
  * x            (x) float64 2kB 3.095e+06 3.095e+06 ... 3.097e+06 3.097e+06
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables: (12/15)
    nir          (time, y, x) uint16 67kB dask.array<chunksize=(1, 133, 253), meta=np.ndarray>
    red          (time, y, x) uint16 67kB dask.array<chunksize=(1, 133, 253), meta=np.ndarray>
    blue         (time, y, x) uint16 67kB dask.array<chunksize=(1, 133, 253), meta=np.ndarray>
    green        (time, y, x) uint16 67kB dask.array<chunksize=(1, 133, 253), meta=np.ndarray>
    emad         (time, y, x) float32 135kB dask.array<chunksize=(1, 133, 253), meta=np.ndarray>
    smad         (time, y, x) float32 135kB dask.array<chunksize=(1, 133, 253), meta=np.ndarray>
    ...           ...
    swir16       (time, y, x) uint16 67kB dask.array<ch

In [9]:
dask_client = DaskClient(n_workers=1, threads_per_worker=16, memory_limit='16GB')
configure_s3_access(cloud_defaults=True, requester_pays=True)

In [10]:
scaled = scale(data)
scaled = scaled.compute().squeeze()

In [11]:
# Explore the site we are working on
scaled.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

In [12]:
scaled

<xarray.Dataset> Size: 4MB
Dimensions:      (y: 133, x: 253)
Coordinates:
  * y            (y) float64 1kB -2.052e+06 -2.052e+06 ... -2.054e+06 -2.054e+06
  * x            (x) float64 2kB 3.095e+06 3.095e+06 ... 3.097e+06 3.097e+06
    spatial_ref  int32 4B 3832
    time         datetime64[ns] 8B 2024-01-01
Data variables: (12/15)
    nir          (y, x) float64 269kB 0.0516 0.0472 0.044 ... 0.0369 0.0341
    red          (y, x) float64 269kB 0.0772 0.0739 0.071 ... 0.0545 0.0532
    blue         (y, x) float64 269kB 0.0826 0.0815 0.0794 ... 0.0813 0.0796
    green        (y, x) float64 269kB 0.1042 0.1005 0.0986 ... 0.09 0.0894
    emad         (y, x) float32 135kB 0.1212 0.115 0.1106 ... 0.1006 0.09127
    smad         (y, x) float32 135kB 4.39e-06 4.4e-06 ... 3.144e-06 3.165e-06
    ...           ...
    swir16       (y, x) float64 269kB 0.0192 0.019 0.0198 ... 0.031 0.031 0.0289
    swir22       (y, x) float64 269kB 0.0177 0.0175 0.018 ... 0.029 0.0271
    coastal      (y, x) float64 269kB 0.0657 0.0658 0.0658 ... 0.0766 0.074
    rededge1     (y, x) float64 269kB 0.0958 0.0958 0.0888 ... 0.0562 0.0533
    rededge2     (y, x) float64 269kB 0.049 0.049 0.0453 ... 0.0367 0.0344
    rededge3     (y, x) float64 269kB 0.0544 0.0543 0.0505 ... 0.0385 0.0361

In [13]:
scaled = calculate_band_indices(scaled)
Dataset = scaled

### GLCM texture analysis

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [14]:
WINDOW_SIZE = 9
LEVELS = 32

# Input
max = scaled.blue.max().values
min = scaled.blue.min().values
# Scale to 0-LEVELS for GLCM
img = ((scaled.blue - min) / (max - min) * (LEVELS - 1)).clip(0, LEVELS - 1).values.astype(np.uint8)

# Extract overlapping windows
patches = sliding_window_view(img, (WINDOW_SIZE, WINDOW_SIZE))
# Shape: (rows, cols, win_y, win_x)


In [15]:
import numpy as np # Ensure numpy is imported if not already

# Assuming 'patches' is a 4D NumPy array with dimensions (y_coords, x_coords, window_y_size, window_x_size)
# To get the first patch (at y=0, x=0), you would index it like this:
sample_patch_data = patches[0, 0, :, :]

# Verify the shape of the extracted sample patch data
print(f"Shape of sample_patch_data: {sample_patch_data.shape}")

# Call glcm_features directly on this 2D sample data
sample_result = glcm_features(sample_patch_data)

# Print the shape of the result to get the number of features
print(f"Shape of glcm_features output for a single patch: {sample_result.shape}")

Shape of sample_patch_data: (9, 9)
Shape of glcm_features output for a single patch: (7,)


In [16]:
# Use apply_ufunc to vectorize over (row, col) dimensions
result = xr.apply_ufunc(
    glcm_features,
    xr.DataArray(patches, dims=["y", "x", "win_y", "win_x"]),
    input_core_dims=[["win_y", "win_x"]],
    output_core_dims=[["feature"]],
    vectorize=True,
    dask="parallelized",
    output_dtypes=[np.float32]
)

# Add coordinates & names
pad = WINDOW_SIZE - 1
result = result.assign_coords({
    "y": scaled.y[: -pad],
    "x": scaled.x[: -pad],
    "feature": ["contrast", "homogeneity", "energy", "ASM", "correlation", "mean", "entropy"]
})

result_bands = result.to_dataset(dim="feature")

# Combine with original
combined = scaled.copy()
combined = combined.assign(result_bands)

combined

<xarray.Dataset> Size: 8MB
Dimensions:      (y: 133, x: 253)
Coordinates:
  * y            (y) float64 1kB -2.052e+06 -2.052e+06 ... -2.054e+06 -2.054e+06
  * x            (x) float64 2kB 3.095e+06 3.095e+06 ... 3.097e+06 3.097e+06
    spatial_ref  int32 4B 3832
    time         datetime64[ns] 8B 2024-01-01
Data variables: (12/34)
    nir          (y, x) float64 269kB 0.0516 0.0472 0.044 ... 0.0369 0.0341
    red          (y, x) float64 269kB 0.0772 0.0739 0.071 ... 0.0545 0.0532
    blue         (y, x) float64 269kB 0.0826 0.0815 0.0794 ... 0.0813 0.0796
    green        (y, x) float64 269kB 0.1042 0.1005 0.0986 ... 0.09 0.0894
    emad         (y, x) float32 135kB 0.1212 0.115 0.1106 ... 0.1006 0.09127
    smad         (y, x) float32 135kB 4.39e-06 4.4e-06 ... 3.144e-06 3.165e-06
    ...           ...
    homogeneity  (y, x) float32 135kB 0.8542 0.8472 0.8403 ... nan nan nan
    energy       (y, x) float32 135kB 0.5446 0.5341 0.5251 ... nan nan nan
    ASM          (y, x) float32 135kB 0.2966 0.2853 0.2758 ... nan nan nan
    correlation  (y, x) float32 135kB 0.487 0.484 0.4523 0.3894 ... nan nan nan
    mean         (y, x) float32 135kB 4.354 4.347 4.382 4.431 ... nan nan nan
    entropy      (y, x) float32 135kB 2.042 2.095 2.1 2.049 ... nan nan nan nan

In [17]:
masked_combined, mask = all_masks(combined, return_mask = True)
mask.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

In [18]:
masked_combined.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

/srv/conda/envs/notebook/lib/python3.10/site-packages/odc/geo/_rgba.py:55: RuntimeWarning: invalid value encountered in cast
  return x.astype("uint8")


### Postcard csv

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [19]:
# Reproject training data to the GeoMAD CRS and convert to xarray
training_reprojected = training.to_crs(masked_combined.odc.crs)
training_da = training_reprojected.assign(
    x=training_reprojected.geometry.x, y=training_reprojected.geometry.y
).to_xarray()

# Extract training values from the masked dataset
training_values = (
    masked_combined.sel(training_da[["x", "y"]], method="nearest")
    .squeeze()
    .compute()
    .to_pandas()
)
training_values

,nir,red,blue,green,emad,smad,bcmad,nir08,nir09,swir16,...,homogeneity,energy,ASM,correlation,mean,entropy,y,x,spatial_ref,time
index,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2052695.0,3095645.0,3832,2024-01-01
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2052955.0,3095645.0,3832,2024-01-01
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2053215.0,3095545.0,3832,2024-01-01
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2053465.0,3095545.0,3832,2024-01-01
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2053675.0,3096095.0,3832,2024-01-01
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2053565.0,3096285.0,3832,2024-01-01
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2053545.0,3095145.0,3832,2024-01-01
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2053275.0,3094865.0,3832,2024-01-01
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2053205.0,3094605.0,3832,2024-01-01


In [20]:
# Join the training data with the extracted values and remove unnecessary columns
training_array = pd.concat([training["cc_id"], training_values], axis=1)

# Drop rows where there was no data available
training_array = training_array.dropna()

# Preview our resulting training array
training_array.head()

,cc_id,nir,red,blue,green,emad,smad,bcmad,nir08,nir09,...,homogeneity,energy,ASM,correlation,mean,entropy,y,x,spatial_ref,time
18,7.0,0.0817,0.0581,0.0658,0.0714,0.145642,0.000007,0.000032,0.0713,0.0568,...,0.527985,0.195434,0.038194,0.725821,4.923611,5.135154,-2053065.0,3095445.0,3832,2024-01-01
19,7.0,0.0633,0.0609,0.0696,0.0785,0.128083,0.000005,0.000028,0.0579,0.0492,...,0.652778,0.171515,0.029417,0.979090,9.583333,5.375300,-2053075.0,3095295.0,3832,2024-01-01
20,7.0,0.0844,0.0838,0.0889,0.0992,0.156817,0.000004,0.000027,0.0772,0.0469,...,0.649428,0.185561,0.034433,0.939331,7.812500,5.152138,-2053005.0,3095055.0,3832,2024-01-01
21,7.0,0.0579,0.0633,0.0700,0.0786,0.091805,0.000004,0.000020,0.0541,0.0331,...,0.602130,0.194196,0.037712,0.927337,9.159722,5.585292,-2052895.0,3094825.0,3832,2024-01-01
22,7.0,0.0716,0.0659,0.0733,0.0814,0.140231,0.000006,0.000028,0.0633,0.0482,...,0.640278,0.235498,0.055459,0.953526,7.076389,4.728574,-2052765.0,3094655.0,3832,2024-01-01


In [21]:
print(training_array.shape[1], 'total columns')
print('columns included', training_array.columns)

39 total columns
columns included Index(['cc_id', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
       'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1', 'rededge2',
       'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'evi', 'savi', 'ndwi',
       'b_g', 'b_r', 'mci', 'ndci', 'ln_bg', 'contrast', 'homogeneity',
       'energy', 'ASM', 'correlation', 'mean', 'entropy', 'y', 'x',
       'spatial_ref', 'time'],
      dtype='object')


In [22]:
standard_schema = ['cc_id', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
       'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1',
       'rededge2', 'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'evi', 'savi',
       'ndwi', 'b_g', 'b_r', 'mci', 'ndci', 'ln_bg', 'contrast', 'homogeneity',
       'energy', 'ASM', 'correlation', 'mean', 'entropy', 'y', 'x', 'time',
       'spatial_ref']

In [23]:
training_array=training_array[standard_schema]

In [24]:
training_array=training_array.drop(columns=["spatial_ref", "time"])

In [25]:
# Write the training data to a CSV file
training_array.to_csv(f"training-data/csvs/{version}-training.csv", index=False)

In [26]:
training_array["cc_id"].dtype

dtype('float64')